### IMPORT LIBRARY

In [1]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event
import pyodbc
import requests
import inspect
from validate_update import *
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from termcolor import colored
from send_email import *

### CONNECT TO AZURE SQL

In [2]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

#Test Connection
try:
    result = engine.execute("SELECT 1")
    print("CONNECTION SUCESSFUL!")
except Exception as e:
    print("CONNECTION FAILED:",str(e))

C:\Users\hcmcbi-intern04\AppData\Local\Temp\ipykernel_11700\723339813.py:12: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  result = engine.execute("SELECT 1")


CONNECTION SUCESSFUL!


### CONNECT TO SHAREPOINT

In [3]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
config_BIHub
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

Web site title: S22M Research & BI Hub _ Successful Connection!


### READ AND LIST FOLDER

In [4]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Binh Duong Province
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Dong Nai Province
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Long An Province
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Hung Yen Province


In [5]:
#lấy năm
list_folder_sub1=[]
for i in list_folder:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub1=list_folder_sub1+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Binh Duong Province/2023
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Binh Duong Province/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Dong Nai Province/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2023
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City/2023
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Long An Province/2022
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Hung Yen Province/2022


In [6]:
#lấy quý
list_folder_sub2=[]
for i in list_folder_sub1:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub2=list_folder_sub2+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Binh Duong Province/2023/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Binh Duong Province/2022/Q3
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Dong Nai Province/2022/Q3
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2023/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2022/Q3
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2022/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City/2023/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City/2022/Q3
/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ha Noi City/2022/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Long An P

In [7]:
#Provide information
Province_name=['Ho Chi Minh City']#Chọn tỉnh cần import-- là tên folder 
Year=['2023']#Chọn năm cần import
Quarter=['Q1']#Chọn quý cần import
#Có thể chọn nhiều tỉnh, tên tỉnh để trong dấu nháy đơn và cách nhau bởi dấy phẩy. Áp dụng tương tự cho năm, quý
#-------------------------------------------------------
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder_sub2:
    if i.split('/')[-3] in Province_name and i.split('/')[-2] in Year and i.split('/')[-1] in Quarter:
        df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_url = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2023/Q1/Update_APT_HCM_20230331.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2023/Q1/Update_SA_HCM_20230331.xlsx


### GET IMPORT FILE

In [8]:
url = []
for i in list_url:
    sector = i.split('/')[-1].split('_')[1].upper()
    if sector in ('APT'):#Chọn sector cần import: RETAIL, OFFICE, SA, HOTEL, APT, VLTH
        url.append(i)
    else:
        pass
url

['/sites/BIHub/Shared Documents/Advisory Data/Main sector/Update/Dim/Ho Chi Minh City/2023/Q1/Update_APT_HCM_20230331.xlsx']

### IMPORT DATA

In [9]:
'''Prepare ingredients''' 
columns_that_need_unidecode = ['Sub_Project_Name', 'Project_District_Name', 'Project_City_Name'
                               , 'New_Sub_Project_Name', 'New_Project_District_Name', 'New_Project_City_Name'
                              ]
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
name_sector = ['RETAIL', 'SA', 'APT', 'VLTH', 'OFFICE', 'HOTEL']
for i in name_sector:
    globals()[f'df_update_{i.lower()}'] = pd.DataFrame([])
#-------------------------------------------------------
'''Get data''' 
for file_url in tqdm(url):  
    data, file_name, sector = get_data(relative_url, file_url)
    #-------------------------------------------------------
    '''Validation step'''
    #Fill up if lười điền =))
    data['New_Sub_Project_Name']= np.where(data['New_Sub_Project_Name'].isnull(), data['Sub_Project_Name'], data['New_Sub_Project_Name'])
    #Remove unfortmated values
    data = remove_unformated_character(data)
    #Remove unicode characters
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])
    #Check dictionary
    lst_dict = ['City', 'District', 'Grade', 'Type']
    for p in lst_dict:
        lst_p = [x for x in data.columns[data.columns.str.contains(p)]]
        for i, j in zip([x for x in data[lst_p] if 'New' not in x], [x for x in data[lst_p] if 'New' in x]):
            data[j] = data[j].fillna(data[i])
        for i in lst_p:
            data, df_dict = check_dictionary(df_dict, file_name, data, i, p, sector, engine, sp_object)
    if len(df_dict) == 0:
        print(colored('Validate succesfully','green'))
        #-------------------------------------------------------
        '''Import data process'''
        #Check project key
        processed_data, flag_key = check_project_key(file_url, data, sector, engine)
        new_processed_data, flag_new_key = check_new_key(file_url, data, sector, engine)
        if flag_key == 0 and flag_new_key ==0:
            data = get_project_key(flag_key, processed_data, data, sector, engine)
            data = get_new_key(flag_new_key, new_processed_data, data, sector, engine)
            if sector == 'RETAIL':
                df_update_retail = pd.concat([df_update_retail, data], axis=0)
            elif sector == 'OFFICE':
                df_update_office = pd.concat([df_update_office, data], axis=0)
            elif sector == 'HOTEL':
                df_update_hotel = pd.concat([df_update_hotel, data], axis=0)
            elif sector == 'SA' or sector=='SERVICED_APARTMENT':
                df_update_sa = pd.concat([df_update_sa, data], axis=0)
            elif sector == 'APT' or sector=='APARTMENT':
                df_update_apt = pd.concat([df_update_apt, data], axis=0)
            elif sector == 'VLTH':
                df_update_vlth = pd.concat([df_update_vlth, data], axis=0)
        else:
            if flag_key != 0:
                print(colored(f'Cant get key in {file_name}', 'yellow'))
            elif flag_new_key != 0:
                print(colored(f'Cant get new key in {file_name}', 'yellow'))

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Validate succesfully


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.12s/it]


In [10]:
df_update_retail

""
